In [ ]:
import time

def binary_mapper(col, lower, upper):
    if col == None:
        return 0
    if col < lower:
        return -1
    elif col > upper:
        return 1
    else:
        return 0

def one_factor_calculate( df ):
    lower = df.RF_Score.quantile(0.1)
    upper = df.RF_Score.quantile(0.9)
    confidenceValue = df['RF_Score'].apply(lambda x:binary_mapper(x,lower,upper))
    return pd.Series(np.sum(confidenceValue*df['returnsOpenNextMktres10']))

def one_factor_eval( df ):
    one_factor_returns = df.groupby('time').apply( one_factor_calculate )
    return float(one_factor_returns.mean() / one_factor_returns.std())

'''
@ param X, y are pandas dataframe
@ return nothing
'''
def time_series_cross_validation(X, y):
    from sklearn.ensemble import RandomForestClassifier
    X = np.array(X); y = np.array(y)
    print("Data Shape  :", X.shape, y.shape)
    from sklearn.model_selection import TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)
    X = np.array(X)
    print(tscv)  
    TimeSeriesSplit(max_train_size=None, n_splits=5)
    cv_result = []
    for train_index, test_index in tscv.split(X):
        print("TRAIN length:", len(train_index), "TEST length:", len(test_index))
        clf = RandomForestClassifier(n_estimators=100, max_depth = 3)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        y_test_pred = clf.predict(X_test)
        y_pred = [i[1] for i in clf.predict_proba(X)]
        from sklearn.metrics import accuracy_score
        print("--- out of sample accuracy ---:", accuracy_score(y_train, clf.predict(X_train)))
        print("--- out of sample accuracy ---:", accuracy_score(y_test, y_test_pred))
        from sklearn.metrics import classification_report
        target_names = ['short', "long"]
        print(" --- classification report ---:\n", classification_report(y_test, y_test_pred, target_names=target_names))
        assert(len(market_train_df) == len(X))
        market_train_df['RF_Score'] = y_pred
        sharpe = one_factor_eval(market_train_df)
        print("sharpe ratio:", sharpe)
        print(market_train_df.columns)
        cv_result.append(sharpe)
        print ('--- %s seconds ---' % (time.time() - start_time))
        
start_time = time.time()
time_series_cross_validation(X, y)
print ('--- %s seconds ---' % (time.time() - start_time))